In [1]:
from kingdon import Algebra

In [2]:
alg = Algebra(2, 0, 1)
u = alg.multivector(name='u')
v = alg.multivector(name='v')

In [10]:
cayley = alg.cayley.copy()
cayley

{('e', 'e'): 'e',
 ('e', 'e0'): 'e0',
 ('e', 'e1'): 'e1',
 ('e', 'e2'): 'e2',
 ('e', 'e01'): 'e01',
 ('e', 'e02'): 'e02',
 ('e', 'e12'): 'e12',
 ('e', 'e012'): 'e012',
 ('e0', 'e'): 'e0',
 ('e0', 'e0'): '0',
 ('e0', 'e1'): 'e01',
 ('e0', 'e2'): 'e02',
 ('e0', 'e01'): '0',
 ('e0', 'e02'): '0',
 ('e0', 'e12'): 'e012',
 ('e0', 'e012'): '0',
 ('e1', 'e'): 'e1',
 ('e1', 'e0'): '-e01',
 ('e1', 'e1'): 'e',
 ('e1', 'e2'): 'e12',
 ('e1', 'e01'): '-e0',
 ('e1', 'e02'): '-e012',
 ('e1', 'e12'): 'e2',
 ('e1', 'e012'): '-e02',
 ('e2', 'e'): 'e2',
 ('e2', 'e0'): '-e02',
 ('e2', 'e1'): '-e12',
 ('e2', 'e2'): 'e',
 ('e2', 'e01'): 'e012',
 ('e2', 'e02'): '-e0',
 ('e2', 'e12'): '-e1',
 ('e2', 'e012'): 'e01',
 ('e01', 'e'): 'e01',
 ('e01', 'e0'): '0',
 ('e01', 'e1'): 'e0',
 ('e01', 'e2'): 'e012',
 ('e01', 'e01'): '0',
 ('e01', 'e02'): '0',
 ('e01', 'e12'): 'e02',
 ('e01', 'e012'): '0',
 ('e02', 'e'): 'e02',
 ('e02', 'e0'): '0',
 ('e02', 'e1'): '-e012',
 ('e02', 'e2'): 'e0',
 ('e02', 'e01'): '0',
 ('e02',

## 修正Cayley表

Kingdon中使用`e02`为基，而PGA Cheatsheet中使用`e20`。以后者为准。

In [22]:
def negate(value):
    if value == "0":
        return value
    elif value[0] == "-":
        return value[1:]
    else:
        return "-" + value
    
def fix(value):
    if value == "e02":
        return "-e20"
    elif value == "-e02":
        return "e20"
    else:
        return value
    
patched_cayley = {}

# patch keys
for key, value in cayley.items():
    if key == ("e02", "e02"):
        patched_cayley[("e20", "e20")] = "0"
    elif key[0] == "e02":
        patched_cayley[("e20", key[1])] = negate(value)
    elif key[1] == "e02":
        patched_cayley[(key[0], "e20")] = negate(value)
    else:
        patched_cayley[key] = value

# patch values
for key, value in patched_cayley.items():
    patched_cayley[key] = fix(value)

patched_cayley

{('e', 'e'): 'e',
 ('e', 'e0'): 'e0',
 ('e', 'e1'): 'e1',
 ('e', 'e2'): 'e2',
 ('e', 'e01'): 'e01',
 ('e', 'e20'): 'e20',
 ('e', 'e12'): 'e12',
 ('e', 'e012'): 'e012',
 ('e0', 'e'): 'e0',
 ('e0', 'e0'): '0',
 ('e0', 'e1'): 'e01',
 ('e0', 'e2'): '-e20',
 ('e0', 'e01'): '0',
 ('e0', 'e20'): '0',
 ('e0', 'e12'): 'e012',
 ('e0', 'e012'): '0',
 ('e1', 'e'): 'e1',
 ('e1', 'e0'): '-e01',
 ('e1', 'e1'): 'e',
 ('e1', 'e2'): 'e12',
 ('e1', 'e01'): '-e0',
 ('e1', 'e20'): 'e012',
 ('e1', 'e12'): 'e2',
 ('e1', 'e012'): 'e20',
 ('e2', 'e'): 'e2',
 ('e2', 'e0'): 'e20',
 ('e2', 'e1'): '-e12',
 ('e2', 'e2'): 'e',
 ('e2', 'e01'): 'e012',
 ('e2', 'e20'): 'e0',
 ('e2', 'e12'): '-e1',
 ('e2', 'e012'): 'e01',
 ('e01', 'e'): 'e01',
 ('e01', 'e0'): '0',
 ('e01', 'e1'): 'e0',
 ('e01', 'e2'): 'e012',
 ('e01', 'e01'): '0',
 ('e01', 'e20'): '0',
 ('e01', 'e12'): '-e20',
 ('e01', 'e012'): '0',
 ('e20', 'e'): 'e20',
 ('e20', 'e0'): '0',
 ('e20', 'e1'): 'e012',
 ('e20', 'e2'): '-e0',
 ('e20', 'e01'): '0',
 ('e20', '

### 生成分级几何乘法代码

In [37]:
grades = [
    ["e"],
    ["e0", "e1", "e2"],
    ["e01", "e20", "e12"],
    ["e012"]
]

def product(ga, gb, name_a="a", name_b="b", name_r="r"):
    grade_a = grades[ga]
    grade_b = grades[gb]

    items = {}

    for basis_a in grade_a:
        for basis_b in grade_b:
            item = "{}.{} * {}.{}".format(name_a, basis_a, name_b, basis_b)
            basis_r = patched_cayley[(basis_a, basis_b)]
            if basis_r not in items:
                items[basis_r] = []

            items[basis_r].append(item)
    return items

product(2,2)

{'0': ['a.e01 * b.e01', 'a.e01 * b.e20', 'a.e20 * b.e01', 'a.e20 * b.e20'],
 '-e20': ['a.e01 * b.e12'],
 'e01': ['a.e20 * b.e12'],
 'e20': ['a.e12 * b.e01'],
 '-e01': ['a.e12 * b.e20'],
 '-e': ['a.e12 * b.e12']}

In [52]:
items = product(2, 2)

def collect(items):
    lines = []
    for basis in [basis for grade in grades for basis in grade]:
        if basis in items:
            line = ""
            line += " + ".join(items[basis])
            if line != "":
                line = "r.{} += {}".format(basis, line)
                lines.append(line)
        if "-" + basis in items:
            line = ""
            line += " - ".join(items["-" + basis])
            if line != "":
                line = "r.{} -= {}".format(basis, line)
                lines.append(line)
        

    code = "\n".join(lines)
    return code

code = collect(items)
print(code)

r.e -= a.e12 * b.e12
r.e01 += a.e20 * b.e12
r.e01 -= a.e12 * b.e20
r.e20 += a.e12 * b.e01
r.e20 -= a.e01 * b.e12


In [54]:
for i in range(4):
    for j in range(4):
        items = product(i, j)
        code = collect(items)
        print("product{}{}".format(i, j))
        print(code)

product00
r.e += a.e * b.e
product01
r.e0 += a.e * b.e0
r.e1 += a.e * b.e1
r.e2 += a.e * b.e2
product02
r.e01 += a.e * b.e01
r.e20 += a.e * b.e20
r.e12 += a.e * b.e12
product03
r.e012 += a.e * b.e012
product10
r.e0 += a.e0 * b.e
r.e1 += a.e1 * b.e
r.e2 += a.e2 * b.e
product11
r.e += a.e1 * b.e1 + a.e2 * b.e2
r.e01 += a.e0 * b.e1
r.e01 -= a.e1 * b.e0
r.e20 += a.e2 * b.e0
r.e20 -= a.e0 * b.e2
r.e12 += a.e1 * b.e2
r.e12 -= a.e2 * b.e1
product12
r.e0 += a.e2 * b.e20
r.e0 -= a.e1 * b.e01
r.e1 -= a.e2 * b.e12
r.e2 += a.e1 * b.e12
r.e012 += a.e0 * b.e12 + a.e1 * b.e20 + a.e2 * b.e01
product13
r.e01 += a.e2 * b.e012
r.e20 += a.e1 * b.e012
product20
r.e01 += a.e01 * b.e
r.e20 += a.e20 * b.e
r.e12 += a.e12 * b.e
product21
r.e0 += a.e01 * b.e1
r.e0 -= a.e20 * b.e2
r.e1 += a.e12 * b.e2
r.e2 -= a.e12 * b.e1
r.e012 += a.e01 * b.e2 + a.e20 * b.e1 + a.e12 * b.e0
product22
r.e -= a.e12 * b.e12
r.e01 += a.e20 * b.e12
r.e01 -= a.e12 * b.e20
r.e20 += a.e12 * b.e01
r.e20 -= a.e01 * b.e12
product23
r.e0 -= 